In [1]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
from fake_useragent import UserAgent

Для сбора отзывов о фильмах используем функции из Parser.ipynb.

In [3]:
def get_page_content(URL):
    """Функция для отправления запроса на страницу и парсинга BeautifulSoup"""
    response = requests.get(URL, headers={'User-Agent': UserAgent().safari})
    if not response.ok:
        return response
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup

In [4]:
def get_genres_list(response_content):
    """Функция для получения ссылок на страницы жанров"""
    genres = []
    obj = response_content.find('ul', attrs = {'class':'zhanr'})

    for item in obj.children:
        try:    
            for inner_item in item.children:
                genres.append(inner_item.get("href"))
        except AttributeError:
            pass
    return genres

In [5]:
def get_links_list(response_content):
    """Функция для получения списка ссылок на страницы фильмов"""
    obj = response_content.find('div', attrs = {'class':'art content'})
    content = obj.find_all('a')

    page_list = []
    for item in content:
        if item.get("href").startswith('https://doramy.club/'):
            try:
                href = item.get("href")
                if '/genre/' not in href:
                    page_list. append(href)
            except AttributeError:
                pass
        
    return page_list

In [9]:
def create_data_file(file_name):
    """Инициализирует пустой файл для сбора данных"""
    columns = '!;!'.join(['Name', 'Reviews\n'])
    with open(file_name, 'w') as file:
        file.write(columns)

def add_info_to_DF(file_name, data):
    """Добаление записи в файл данных"""
    with open(file_name, 'a') as file:
        file.write(data)
        
def get_last_page_number(response_content):
    """Получение номера последней страницы на странице жанра"""
    obj = response_content.find('div', attrs = {'class':'art content'})
    content = obj.find_all('a')  
    return int(content[-2].text)

In [11]:
def get_reviews(links, data, pause):
    """Функция для прохода по списку ссылок на страницы фильмов и сбора данных со страницы"""
    for link in links:
        if link.startswith('https://doramy.club/') and 'page' not in link:
            time.sleep(pause)
            print('Скачиваем', link)
            URL = link
            source = get_page_content(URL)
            try:
                obj = source.find_all('div', attrs = {'class':'ct-text clearfix'})
                name = source.find('h1', attrs = {'itemprop':'name'}).text
                string = '!;!'.join([name, ' '.join([item.text for item in obj]).replace('\n', ' ').strip('\n')])
            except Exception:
                print('\n','-'*10)
                print('Проблема со скачиванием:', link)
                print('\n','-'*10)
                pass
            data += string + '\n'
    return data

Теперь соберем отзывы пользователей.

In [12]:
page = get_page_content('https://doramy.club/')
genres_list = get_genres_list(page)

In [13]:
pause = 5
file_name = 'results.csv'
create_data_file(file_name)
for genre in genres_list:
    time.sleep(pause)
    print('\nНачинаем скачивать', genre)
    URL = 'https://doramy.club/'[:-1] + genre
    source = get_page_content(URL)    
    last_page_number = get_last_page_number(source)
    links = get_links_list(source)
    
    data = ''
    data = get_reviews(links, data, pause)
    k = 2
    for page_number in range(k, last_page_number + 1):  
        time.sleep(pause)
        print('Скачиваем страницу', page_number, '\n')
        URL = 'https://doramy.club/'[:-1] + genre + '/page/{}'.format(page_number)
        source = get_page_content(URL)
        links = get_links_list(source)
        data = get_reviews(links, data, pause)

        if page_number % 5 == 0:
            print('-'*10)
            add_info_to_DF(file_name, data)
            data = ''


Начинаем скачивать /genre/boevik
Скачиваем https://doramy.club/37165-c-toboj-v-den-konca-sveta-4.html
Скачиваем https://doramy.club/34949-iz-za-lyubvi.html
Скачиваем https://doramy.club/28944-slyozy-v-ogne.html
Скачиваем https://doramy.club/36289-dvojnaya-dikost.html
Скачиваем https://doramy.club/37704-oxotnichi-psy.html
Скачиваем https://doramy.club/37666-vlyublennye-vo-vremeni-i-prostranstve.html
Скачиваем https://doramy.club/36505-semya.html
Скачиваем https://doramy.club/31560-vedma-2-drugaya.html
Скачиваем https://doramy.club/31774-chest-policii.html
Скачиваем https://doramy.club/33770-svet-zhizni.html
Скачиваем страницу 24 

Скачиваем https://doramy.club/17447-xuan-fej-xun.html
Скачиваем https://doramy.club/17514-xitmen.html
Скачиваем https://doramy.club/7454-s-c-i-kollekciya-zagadochnyx-del.html
Скачиваем https://doramy.club/1746-u-sin-izgonyayushhij-demonov-2.html
Скачиваем https://doramy.club/12775-serdce-rycarya.html
Скачиваем https://doramy.club/14818-legenda-o-razorenii-din

Скачиваем https://doramy.club/2815-bratstvo-klinkov-2.html
Скачиваем https://doramy.club/2441-real.html
Скачиваем https://doramy.club/1824-zlodejka.html
Скачиваем https://doramy.club/1597-rasxititeli-vremeni-zateryannaya-grobnica.html
Скачиваем страницу 37 

Скачиваем https://doramy.club/1392-moczin-zabytaya-legenda.html
Скачиваем https://doramy.club/1295-ajris.html
Скачиваем https://doramy.club/1173-teloxranitel.html
Скачиваем https://doramy.club/970-okcha.html
Скачиваем https://doramy.club/790-dve-nedeli.html
Скачиваем https://doramy.club/742-vy-okruzheny.html
Скачиваем https://doramy.club/249-licom-k-licu.html
Скачиваем https://doramy.club/738-sotrudnichestvo.html
Скачиваем https://doramy.club/725-ekstraordinarnaya-missiya.html
Скачиваем https://doramy.club/712-chosonskij-strelok.html
Скачиваем страницу 38 

Скачиваем https://doramy.club/582-mstitel-v-maske.html
Скачиваем https://doramy.club/586-bozhestvennyj-geroj.html
Скачиваем https://doramy.club/567-beglec-plan-b.html
Скачиваем 

Скачиваем https://doramy.club/2150-ukun-car-obezyan.html
Скачиваем https://doramy.club/2139-bratstvo-klinkov.html
Скачиваем https://doramy.club/1593-oxotniki-za-sokrovishhami-2.html
Скачиваем https://doramy.club/908-general-i-ya.html
Скачиваем https://doramy.club/1092-dom-letayushhix-kinzhalov.html
Скачиваем https://doramy.club/773-oxotniki-za-sokrovishhami.html
Скачиваем https://doramy.club/548-teloxranitel-shkolnoj-krasavicy.html
Скачиваем страницу 12 

Скачиваем https://doramy.club/484-tajna-blestyashhego-kamnya.html
Скачиваем https://doramy.club/420-il-chzhi-me.html
Скачиваем https://doramy.club/314-propavshij-master-mecha.html

Начинаем скачивать /genre/voennyj
Скачиваем https://doramy.club/15452-imperatorskoe-sovershenstvo.html
Скачиваем https://doramy.club/36200-bezymyannyj.html
Скачиваем https://doramy.club/35972-obyazannosti-posle-shkoly.html
Скачиваем https://doramy.club/35371-liniya-fronta.html
Скачиваем https://doramy.club/29527-kozyrnye-vojska.html
Скачиваем https://doramy

Скачиваем страницу 7 

Скачиваем https://doramy.club/35735-prikosnis-ko-mne-esli-smozhesh.html
Скачиваем https://doramy.club/34441-moj-napolovinu-paren.html
Скачиваем https://doramy.club/35642-svadba-s-vladykoj-carstva-tenej.html
Скачиваем https://doramy.club/35632-eyo-vysochestvo.html
Скачиваем https://doramy.club/34653-davaj-srazhatsya.html
Скачиваем https://doramy.club/32850-lyubov-nachinaetsya-s-braka.html
Скачиваем https://doramy.club/32557-moya-devushka-znaet-kung-fu.html
Скачиваем https://doramy.club/35310-nash-den-d.html
Скачиваем https://doramy.club/30169-luchshaya-oshibka-3.html
Скачиваем https://doramy.club/33302-podobnaya-cvetku-podobnyj-osotu.html
Скачиваем страницу 8 

Скачиваем https://doramy.club/33947-malenkaya-svaxa-imperatora-demonov.html
Скачиваем https://doramy.club/30614-ya-tvoj-paren.html
Скачиваем https://doramy.club/34650-polyubi-menya-za-tri-dnya.html
Скачиваем https://doramy.club/34727-parallelnaya-missiya.html
Скачиваем https://doramy.club/34616-zagrobnaya-z

Скачиваем https://doramy.club/29426-tolko-ty-2021.html
Скачиваем https://doramy.club/28578-eta-pervaya-lyubov-vydumka.html
Скачиваем https://doramy.club/29416-1-shans-k-10000.html
Скачиваем страницу 20 

Скачиваем https://doramy.club/29328-cvet-slyoz.html
Скачиваем https://doramy.club/29272-rajon-chistoj-melodramy.html
Скачиваем https://doramy.club/9405-1-davno-ne-videlis.html
Скачиваем https://doramy.club/29210-ele-dysha.html
Скачиваем https://doramy.club/28853-aromat-pervogo-cvetka.html
Скачиваем https://doramy.club/28560-akademiya-ubijc.html
Скачиваем https://doramy.club/28180-vy-dostavlyaete-vremya.html
Скачиваем https://doramy.club/24512-vstrecha-u-okna.html
Скачиваем https://doramy.club/29117-segodnya-startuem.html
Скачиваем https://doramy.club/28534-ukus-sestyor.html
----------
Скачиваем страницу 21 

Скачиваем https://doramy.club/29096-obnovlenie-lyubvi.html
Скачиваем https://doramy.club/29088-zazhgite-serdce-mister-vysokomerie.html
Скачиваем https://doramy.club/27910-parni-kot

...

Скачиваем https://doramy.club/27694-gospodin-vozmi-menya-v-zhyony-2.html
Скачиваем https://doramy.club/28841-muki-lyubvi.html
Скачиваем страницу 27 

Скачиваем https://doramy.club/28684-dvorec-kovarnye-zhenshhiny.html
Скачиваем https://doramy.club/24803-1-nefritovoe-serdce-shi.html
Скачиваем https://doramy.club/25646-lyubov-podobna-belomu-nefritu.html
Скачиваем https://doramy.club/28184-ya-stala-porochnoj-aktrisoj-vtorogo-plana.html
Скачиваем https://doramy.club/27785-xon-chxon-gi-alye-nebesa.html
Скачиваем https://doramy.club/27730-koroleva-ataki-2.html
Скачиваем https://doramy.club/26114-vremya-letit-i-ty-zdes.html
Скачиваем https://doramy.club/27604-odin-edinstvennyj.html
Скачиваем https://doramy.club/28361-pak-yol.html
Скачиваем https://doramy.club/24760-ne-ostavlyaj-menya.html
Скачиваем страницу 28 

Скачиваем https://doramy.club/27809-vesennij-son-v-zvyozdnoj-reke.html
Скачиваем https://doramy.club/28342-eliksir-ne-tak-xorosh-kak-ty.html
Скачиваем https://doramy.club/24077-bespod

Скачиваем https://doramy.club/15880-mech-dinastii.html
Скачиваем https://doramy.club/17245-verxovnyj-pravitel.html
Скачиваем https://doramy.club/19996-1-princessa-samozvanka.html
Скачиваем https://doramy.club/21809-dorogoj-povelitel-mediciny.html
Скачиваем страницу 40 

Скачиваем https://doramy.club/23098-ozornaya-princessa.html
Скачиваем https://doramy.club/20852-oj-imperator-vlyublyon.html
Скачиваем https://doramy.club/22944-troecarstvie.html
Скачиваем https://doramy.club/21545-shag-za-shagom-iz-dvorca.html
Скачиваем https://doramy.club/5863-puteshestvie-vo-vremeni-doktora-dzhina.html
Скачиваем https://doramy.club/12019-serebryanaya-princessa.html
Скачиваем https://doramy.club/20579-put-yunosti-toska-lyubvi.html
Скачиваем https://doramy.club/16758-yan-shi-fan-novaya-molodezh.html
Скачиваем https://doramy.club/9507-pesnya-feniksa.html
Скачиваем https://doramy.club/16174-dvorec-yansi-priklyucheniya-princessy.html
----------
Скачиваем страницу 41 

Скачиваем https://doramy.club/17503-ub

Скачиваем https://doramy.club/9806-doch-nalozhnicy-istoriya-min-lan.html
Скачиваем https://doramy.club/11721-ugymtxi.html
Скачиваем https://doramy.club/9461-vnutrennij-dvorec-legenda-o-zhui.html
Скачиваем https://doramy.club/12702-1-novichok-istorik-gu-xe-ryon.html
Скачиваем https://doramy.club/12100-xroniki-asdalya.html
Скачиваем https://doramy.club/12634-neukrotimyj.html
Скачиваем https://doramy.club/7695-davaj-otorvyomsya-2.html
Скачиваем страницу 53 

Скачиваем https://doramy.club/2645-nefritovaya-dinastiya-2.html
Скачиваем https://doramy.club/1495-poddelnye-feniksy.html
Скачиваем https://doramy.club/11554-princessa-chzha-myon-go.html
Скачиваем https://doramy.club/13639-xolostyak-kibana.html
Скачиваем https://doramy.club/12291-proshlaya-zhizn.html
Скачиваем https://doramy.club/11774-1-raznye-mechty.html
Скачиваем https://doramy.club/4227-idealnaya-parochka.html
Скачиваем https://doramy.club/13536-bishunmu-tancuyushhie-v-nebesax.html
Скачиваем https://doramy.club/5059-o-moj-general.

Скачиваем https://doramy.club/2264-don-i.html
Скачиваем https://doramy.club/3343-kundo-epoxa-ugrozy.html
Скачиваем https://doramy.club/1131-princessa-korolya-lan-lin.html
Скачиваем https://doramy.club/1014-lyubov-korolya.html
Скачиваем https://doramy.club/996-tri-zhizni-tri-epoxi-desyat-mil-persikovogo-cveta.html
Скачиваем https://doramy.club/3003-kunxam-pogranichnyj-ostrov.html
Скачиваем https://doramy.club/1341-legenda-o-geroyax-kondorax.html
----------
Скачиваем страницу 66 

Скачиваем https://doramy.club/2815-bratstvo-klinkov-2.html
Скачиваем https://doramy.club/2599-dzhekpot-korolevskij-kush.html
Скачиваем https://doramy.club/2595-1-voin-pek-ton-su.html
Скачиваем https://doramy.club/2589-klinok-i-lepestok-lezvie-i-lepestok.html
Скачиваем https://doramy.club/2139-bratstvo-klinkov.html
Скачиваем https://doramy.club/937-lyubov-skvoz-tysyacheletie.html
Скачиваем https://doramy.club/1157-xudozhnik-dush.html
Скачиваем https://doramy.club/186-saimdan-dnevnik-sveta.html
Скачиваем https://

Скачиваем страницу 10 

Скачиваем https://doramy.club/209-dlya-tebya-vo-vsem-cvetu-yaponskaya-versiya.html
----------

Начинаем скачивать /genre/biografiya
Скачиваем https://doramy.club/33932-snajpery.html
Скачиваем https://doramy.club/31395-ataka-na-pyorl-xarbor.html
Скачиваем https://doramy.club/23551-kak-odin.html
Скачиваем https://doramy.club/23543-admiral-revushhie-potoki.html
Скачиваем https://doramy.club/3103-ovoshhnoj-magazin-xolostyaka.html
Скачиваем https://doramy.club/20287-princessa-dok-xe.html
Скачиваем https://doramy.club/18742-sado.html
Скачиваем https://doramy.club/17984-ispoved-nepolnocennogo-cheloveka.html
Скачиваем https://doramy.club/4044-imperatrica-kitaya.html
Скачиваем https://doramy.club/3441-legendarnyj-magnat.html
Скачиваем страницу 2 

Скачиваем https://doramy.club/9542-odnazhdy-v-kitae.html
Скачиваем https://doramy.club/7242-istoriya-xatiko.html
Скачиваем https://doramy.club/7085-konfucij.html
Скачиваем https://doramy.club/6390-povar-imperatora.html
Скачивае

# Обработка результатов

Удалим дубликаты и объединим данные о фильмах и отзывах на них в один датасет.

In [114]:
cleaned_reviews = pd.read_csv('reviews.csv')

In [115]:
data = pd.read_csv('data.csv')

In [116]:
cleaned_reviews = cleaned_reviews.drop_duplicates()
data = data.drop_duplicates()

In [117]:
cleaned_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5219 entries, 0 to 12261
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Name     5219 non-null   object
 1   Reviews  5198 non-null   object
dtypes: object(2)
memory usage: 122.3+ KB


In [118]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5286 entries, 0 to 12280
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             5286 non-null   object 
 1   Episod_duration  5286 non-null   object 
 2   Total_series     5286 non-null   object 
 3   Year             5286 non-null   int64  
 4   Country          5286 non-null   object 
 5   Genres           5286 non-null   object 
 6   Translation      5286 non-null   object 
 7   Score            5286 non-null   float64
 8   Description      5234 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 413.0+ KB


In [119]:
new_data = data.merge(cleaned_reviews, left_on='Name', right_on='Name' )

In [120]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5325 entries, 0 to 5324
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             5325 non-null   object 
 1   Episod_duration  5325 non-null   object 
 2   Total_series     5325 non-null   object 
 3   Year             5325 non-null   int64  
 4   Country          5325 non-null   object 
 5   Genres           5325 non-null   object 
 6   Translation      5325 non-null   object 
 7   Score            5325 non-null   float64
 8   Description      5273 non-null   object 
 9   Reviews          5305 non-null   object 
dtypes: float64(1), int64(1), object(8)
memory usage: 457.6+ KB


In [121]:
new_data

,Name,Episod_duration,Total_series,Year,Country,Genres,Translation,Score,Description,Reviews
0,Странная легенда династии Тан дорама (2022),45 мин,36 серий,2022,Китай,"Детектив, Исторический, Фэнтези","Русская озвучка, Русские субтитры",9.04,Странные дела происходят в древней столице Ча...,"мне понравилось! Понравилось, но у меня про..."
1,Письмо с небес дорама (2022),44 мин,26 серий,2022,Китай,"Детектив, Исторический, Комедия, Мелодрама",Русские субтитры,9.39,"Молодая, но уже довольно опытная в жизни Шен ...",г героиня просто бесит Озвучьте кто нибудь....
2,Тринадцать лет забвения дорама (2023),44 мин,24 серии,2023,Китай,"Детектив, Криминал, Триллер",Русские субтитры,9.31,Тринадцать лет назад в городе Наньду произошл...,Отличный сериал!!! Игра актеров была на высше...
3,Шах и мат дорама (2022),45 мин,24 серии,2022,Китай,"Детектив, Исторический, Триллер","Русская озвучка, Русские субтитры",8.96,Молодой любитель юриспруденции и расследовани...,"Очень хорошо дорама просто пушка, Очень краси..."
4,Домашняя школа: ученики под арестом дорама (2023),50 мин,18 серий,2023,Таиланд,"Детектив, Триллер","Русская озвучка, Русские субтитры",9.16,"В специальный интернат, заявленный, как домаш...",Полная чушь…! Аааааа ヽ(♡‿♡)ノ Я так долго жд...
...,...,...,...,...,...,...,...,...,...,...
5320,Клуб друзей-водителей дорама (2020),24 мин,9 серий,2020,Таиланд,Ток-шоу,Русские субтитры,9.08,В новом шоу «Клуб друзей-водителей» мы познак...,"В этом шоу 26 серий. Шоу очень приятное, ребя..."
5321,Почтовый ящик БрайтВинов дорама (2020),20 мин,1 серия,2020,Таиланд,Ток-шоу,Русские субтитры,9.29,Популярные парни из Таиланда — Брайт и Вин бу...,Я их обожаю) все дорамы пересмотрела) но у ме...
5322,Прекрасный вид с Сон Джи Хё дорама (2017),54 мин,10 серий,2017,Южная Корея,Ток-шоу,Русские субтитры,8.48,Сон Джи Хё — одна из лучших актрис Южной Коре...,я не знаю но мне нравится shin won ho вы не...
5323,Попался! 2 | Преступник — это ты! 2 дорама (2019),1 час 10 мин.,10 серий,2019,Южная Корея,Ток-шоу,Русские субтитры,8.34,Во втором сезоне детективного ток-шоу «Попалс...,Очень понравился. 100/1000 Омг!!! Шоу потря...


In [124]:
new_data.to_csv('complete_data.csv')